In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# disable warnings
import warnings
warnings.filterwarnings('ignore')

## Filter out small nodes: keep those with at least 10 channels and at least 0.3 btc in capacity

In [3]:
import json
from pandas import json_normalize

In [4]:
# loading the file
file_name = '../Datos/graph_metrics_2022-07-11.json'
graph_data = open(file_name, encoding='utf8')
graph_json = json.load(graph_data)
# converting data to pd dfs
nodes_graph = json_normalize(graph_json['nodes'])
channels_graph = json_normalize(graph_json['edges'])
channels_graph.channel_id = channels_graph.capacity.astype(int)
channels_graph.capacity = channels_graph.capacity.astype(int)

In [5]:
def add_node_chan_info(df_nodes, df_channels):
    df_nodes = pd.concat([
        df_nodes,
        pd.DataFrame(columns=[
                'num_enabled_channels',
                'num_channels',
                'total_node_capacity'
        ])
    ], sort=False)
    
    for index, node in df_nodes.iterrows():

        pub_key = node['pub_key']
        # obtain those channels that were opened by this specific node
        node1_channels = df_channels[df_channels.node1_pub == pub_key]
        
        # obtain those channels that were opened by other nodes
        node2_channels = df_channels[df_channels.node2_pub == pub_key]
        
        total_capacity = (node1_channels['capacity'].sum() + node2_channels['capacity'].sum())
        df_nodes.loc[index, 'total_node_capacity'] = total_capacity
        
        
        # transform False to True and vice versa and make the sum -> this is the enabled channels of a node
        enabled1_channels = (node1_channels['node1_policy.disabled'].transform(lambda x: not x)).sum()
        enabled2_channels = (node2_channels['node2_policy.disabled'].transform(lambda x: not x)).sum()
        
        df_nodes.loc[index, 'num_enabled_channels'] = enabled1_channels + enabled2_channels
        df_nodes.loc[index, 'num_channels'] = node1_channels.shape[0] + node2_channels.shape[0]

    return df_nodes

In [6]:
# network = add_node_chan_info(nodes_graph, channels_graph)

In [7]:
# network.to_csv('../Datos/network.csv', index=False, encoding='utf8')

In [8]:
network = pd.read_csv('../Datos/network.csv')

In [9]:
network.head(2)

,last_update,pub_key,alias,addresses,color,features.0.name,features.0.is_required,features.0.is_known,features.5.name,features.5.is_required,...,features.3.is_known,features.105.name,features.105.is_required,features.105.is_known,features.1339.name,features.1339.is_required,features.1339.is_known,num_enabled_channels,num_channels,total_node_capacity
0,1.657364e+09,020003b9499a97c8dfbbab6b196319db37ba9c37bccb60...,BJCR_BTCPayServer,"[{'network': 'tcp', 'addr': '95.217.192.209:97...",#3399ff,data-loss-protect,True,True,upfront-shutdown-script,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22,23,111713145
1,1.657199e+09,0200072fd301cb4a680f26d87c28b705ccd6a1d5b00f1b...,OutaSpace 🚀,"[{'network': 'tcp', 'addr': '176.28.11.68:9760...",#123456,NaN,NaN,NaN,upfront-shutdown-script,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,7,1893366


In [10]:
filters = (network['num_channels'] >= 10) & (network['total_node_capacity'] >= 30000000)

In [11]:
network[filters].shape

(1809, 101)

In [12]:
nodes = network[filters]

In [13]:
cols = ['pub_key', 'alias' , 'num_channels', 'total_node_capacity']
nodes = nodes[cols]

In [14]:
mask1 = channels_graph['node1_pub'].isin(nodes['pub_key'])
mask2 = channels_graph['node2_pub'].isin(nodes['pub_key'])
channels_graph = channels_graph[mask1 & mask2]

In [15]:
channels_graph

,channel_id,chan_point,last_update,node1_pub,node2_pub,capacity,node1_policy,node2_policy,node1_policy.time_lock_delta,node1_policy.min_htlc,...,node1_policy.disabled,node1_policy.max_htlc_msat,node1_policy.last_update,node2_policy.time_lock_delta,node2_policy.min_htlc,node2_policy.fee_base_msat,node2_policy.fee_rate_milli_msat,node2_policy.disabled,node2_policy.max_htlc_msat,node2_policy.last_update
1092,63874,69cb9da0c31b617a510b06ce6dcb951e862a04c434cbec...,1657478774,03434a39cd9a537c852fc8fb72454086d726f9111e9f73...,03d301eedc0949238bf919452ee7ef5c45bda4adbe17fa...,63874,NaN,NaN,40.0,1000,...,True,63236000,1.657479e+09,144.0,1000,1000,1,False,0,1.532332e+09
1499,1831257,77d828e508f43ade02cb01190021644a9be5ffba5fad27...,1657526194,02445ff6a24e69242d6f6f38e1b7e8ae9c01eb9b660f3d...,029a8741675c4c9078b577ddc4348d602d2fb45a12e608...,1831257,NaN,NaN,144.0,1000,...,False,1812945000,1.657526e+09,144.0,1000,888,1,False,1812945000,1.657526e+09
2796,357883,0bb0411d186af60cea6070695e393606b8810a5edbfdcb...,1657543838,037172d2110c4148d6ed0c2790ec8be948458022425dce...,03beddc8adbf7d56a7da15cdaf95d97b24d07088c3571b...,357883,NaN,NaN,40.0,1000,...,False,354305000,1.648533e+09,18.0,1000000,1000,299,True,354305000,1.657544e+09
2925,2000000,9f0358a76c9a0d06c135007a7d8d15c3397ae82d98decc...,1657520173,02740c62f38896511eb98479036b06907fcac283f62d08...,03beddc8adbf7d56a7da15cdaf95d97b24d07088c3571b...,2000000,NaN,NaN,30.0,1000,...,False,1980000000,1.657520e+09,18.0,1000000,1000,299,False,1980000000,1.657515e+09
2956,1600000,19746a8b45e23766e6cb923971922415c8c21dafa8ea52...,1657532551,02a45def9ae014fdd2603dd7033d157faa3a55a72b06a6...,037172d2110c4148d6ed0c2790ec8be948458022425dce...,1600000,NaN,NaN,24.0,1000,...,True,1584000000,1.657533e+09,40.0,1000,100,120,False,1584000000,1.648535e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85911,15114842,ec976deb404e406fa5f97cd051167906a5ec3fc4c60670...,1657550054,0279f06eba0e1080f6a693201f090d0635a0e5dd2ef57d...,03dab87ff8635982815c4567eb58af48f9944d11c56beb...,15114842,NaN,NaN,34.0,1,...,False,14963694000,1.657549e+09,40.0,1000,2250,55,False,15114842000,1.657550e+09
85912,3000000,ac076bd82418e8387e6bc5f86f177626c7c1b32aaf97bf...,1657551701,02c5dd45bd62dfd948ccb58490e54b8a652f8e8b08ed2f...,02f94c8c175b002c6421f7e97f11f3ce846d2fd7d88a2b...,3000000,NaN,NaN,40.0,1000,...,False,2970000000,1.657552e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85915,50000000,57c3d89c943bd420bd967cb891a2399d243b8b0a072b62...,1657552141,02343c19a39dd11cfc6f7571f36213dd52fb70ee45ee40...,03864ef025fde8fb587d989186ce6a4a186895ee44a926...,50000000,NaN,NaN,40.0,1000,...,False,20000000000,1.657552e+09,144.0,1,1000,499,False,50000000000,1.657552e+09
85916,200000000,3e908ae3c6dc3946a904522642a3ffac5002d2e32380c6...,1657552352,033d8656219478701227199cbd6f670335c8d408a92ae8...,03aab7e9327716ee946b8fbfae039b0db85356549e72c5...,200000000,NaN,NaN,40.0,1000,...,False,198000000000,1.657552e+09,40.0,1000,0,500,False,198000000000,1.657552e+09


In [16]:
channels_graph.columns

Index(['channel_id', 'chan_point', 'last_update', 'node1_pub', 'node2_pub',
       'capacity', 'node1_policy', 'node2_policy',
       'node1_policy.time_lock_delta', 'node1_policy.min_htlc',
       'node1_policy.fee_base_msat', 'node1_policy.fee_rate_milli_msat',
       'node1_policy.disabled', 'node1_policy.max_htlc_msat',
       'node1_policy.last_update', 'node2_policy.time_lock_delta',
       'node2_policy.min_htlc', 'node2_policy.fee_base_msat',
       'node2_policy.fee_rate_milli_msat', 'node2_policy.disabled',
       'node2_policy.max_htlc_msat', 'node2_policy.last_update'],
      dtype='object')

## Data for network

In [17]:
# List of edges: each tuple is an edge (node1, node2)
all_edges = list(zip(channels_graph['node1_pub'], channels_graph['node2_pub']))

In [18]:
# dictionary of sequences: key is attribute name, value is list of values for each edge
all_edge_attrs = {
    'capacity': channels_graph['capacity'].values,
    'node1_fee_rate': channels_graph['node1_policy.fee_rate_milli_msat'].values,
    'node2_fee_rate': channels_graph['node2_policy.fee_rate_milli_msat'].values,
}

In [19]:
ln_capital = '0305f5f4013f6c6eeb097bd8607204ec1f31577a05fae35f0d857c54d3b52e4e45'

In [20]:
okcoin = '036b53093df5a932deac828cca6d663472dbc88322b05eec1d42b26ab9b16caa1c'

In [21]:
wallet = '035e4ff418fc8b5554c5d9eea66396c227bd429a3251c8cbc711002ba215bfc226'

## Build networkx graph

In [22]:
import networkx as nx

In [23]:
G = nx.Graph()

In [24]:
# add edges
G.add_edges_from(all_edges)

In [25]:
dict_atts = {(e[0], e[1]): {'capacity': c, 'node1_fee_rate': n1, 'node2_fee_rate': n2} for e, c, n1, n2 in zip(all_edges, all_edge_attrs['capacity'], all_edge_attrs['node1_fee_rate'], all_edge_attrs['node2_fee_rate'])}

In [26]:
# add edges attributes
nx.set_edge_attributes(G, dict_atts)    

## Get paths

In [27]:
dest_nodes = G.nodes()

In [28]:
source_nodes = G.nodes()

In [29]:
paths = nx.all_simple_paths(G, source=ln_capital, target=okcoin, cutoff=2)

In [30]:
some_paths = []

In [31]:
stop = 0

In [32]:
# for s_node in source_nodes:
for d_node in dest_nodes:
    if not nx.has_path(G, source=ln_capital, target=d_node):
        continue
    for i in range(9):
        stop = 0
        paths = nx.all_simple_paths(G, source=ln_capital, target=d_node, cutoff=i+1)
        while stop < 10:
            try:
                some_paths.append(next(paths))
                stop += 1
            except StopIteration:
                break

In [33]:
# paths from okcoin to ln_capital
paths_okcoin_ln_capital = []
for i in range(9):
    stop = 0
    paths = nx.all_simple_paths(G, source=okcoin, target=ln_capital, cutoff=i+1)
    while stop < 10:
        try:
            paths_okcoin_ln_capital.append(next(paths))
            stop += 1
        except StopIteration:
            break

In [34]:
############################
## testing
############################

In [35]:
test = pd.DataFrame(data={'testing':some_paths})

In [36]:
cum_test = test.explode('testing')

In [37]:
cum_test.reset_index(inplace=True)

In [38]:
cum_test.head(2)

,index,testing
0,0,0305f5f4013f6c6eeb097bd8607204ec1f31577a05fae3...
1,0,0298f6074a454a1f5345cb2a7c6f9fce206cd0bf675d17...


In [39]:
cum_test = cum_test.rename(columns={'testing':'source'})

In [40]:
cum_test['destination'] = cum_test.source.shift(-1)

In [41]:
cum_test.head(4)

,index,source,destination
0,0,0305f5f4013f6c6eeb097bd8607204ec1f31577a05fae3...,0298f6074a454a1f5345cb2a7c6f9fce206cd0bf675d17...
1,0,0298f6074a454a1f5345cb2a7c6f9fce206cd0bf675d17...,03434a39cd9a537c852fc8fb72454086d726f9111e9f73...
2,0,03434a39cd9a537c852fc8fb72454086d726f9111e9f73...,0305f5f4013f6c6eeb097bd8607204ec1f31577a05fae3...
3,1,0305f5f4013f6c6eeb097bd8607204ec1f31577a05fae3...,0260fab633066ed7b1d9b9b8a0fac87e1579d1709e874d...


In [42]:
grouped = cum_test.groupby('index')

In [43]:
all_channels = cum_test[grouped.cumcount(ascending=False) > 0]

In [44]:
all_channels

,index,source,destination
0,0,0305f5f4013f6c6eeb097bd8607204ec1f31577a05fae3...,0298f6074a454a1f5345cb2a7c6f9fce206cd0bf675d17...
1,0,0298f6074a454a1f5345cb2a7c6f9fce206cd0bf675d17...,03434a39cd9a537c852fc8fb72454086d726f9111e9f73...
3,1,0305f5f4013f6c6eeb097bd8607204ec1f31577a05fae3...,0260fab633066ed7b1d9b9b8a0fac87e1579d1709e874d...
4,1,0260fab633066ed7b1d9b9b8a0fac87e1579d1709e874d...,03434a39cd9a537c852fc8fb72454086d726f9111e9f73...
6,2,0305f5f4013f6c6eeb097bd8607204ec1f31577a05fae3...,03a503d8e30f2ff407096d235b5db63b4fcf3f89a653ac...
...,...,...,...
910341,135320,03abf6f44c355dec0d5aa155bdbdd6e0c8fefe318eff40...,03935a378993d0b55056801b11957aaecb9f85f34b6424...
910342,135320,03935a378993d0b55056801b11957aaecb9f85f34b6424...,02b0172bb38617fa3afdb69664468b492d5a21062a4fa8...
910343,135320,02b0172bb38617fa3afdb69664468b492d5a21062a4fa8...,0390b5d4492dc2f5318e5233ab2cebf6d48914881a33ef...
910344,135320,0390b5d4492dc2f5318e5233ab2cebf6d48914881a33ef...,027ce055380348d7812d2ae7745701c9f93e70c1adeb26...


In [45]:
# Need to add capacity and node fee rates

In [46]:
# make capacity and node fee rates columns
all_channels[['capacity', 'node1_fee_rate', 'node2_fee_rate']] = None 

In [47]:
# iterate over all_channels rows and add capacity and node fee rates from G edges
for index, row in all_channels.iterrows():
    all_channels.loc[index, 'capacity'] = G.get_edge_data(row['source'], row['destination'])['capacity']
    all_channels.loc[index, 'node1_fee_rate'] = G.get_edge_data(row['source'], row['destination'])['node1_fee_rate']
    all_channels.loc[index, 'node2_fee_rate'] = G.get_edge_data(row['source'], row['destination'])['node2_fee_rate']

In [48]:
all_channels.head(1)

,index,source,destination,capacity,node1_fee_rate,node2_fee_rate
0,0,0305f5f4013f6c6eeb097bd8607204ec1f31577a05fae3...,0298f6074a454a1f5345cb2a7c6f9fce206cd0bf675d17...,5000000,249,100


In [49]:
all_channels = all_channels.rename(columns={'index':'path_id'})

In [50]:
all_channels.head(1)

,path_id,source,destination,capacity,node1_fee_rate,node2_fee_rate
0,0,0305f5f4013f6c6eeb097bd8607204ec1f31577a05fae3...,0298f6074a454a1f5345cb2a7c6f9fce206cd0bf675d17...,5000000,249,100


In [51]:
aliases = nodes.set_index('pub_key')['alias']

In [52]:
all_channels.set_index('source', inplace=True)
all_channels['s_node'] = aliases

In [53]:
all_channels.reset_index(inplace=True)
all_channels.set_index('destination',inplace=True)
all_channels['d_node'] = aliases

In [54]:
all_channels.reset_index(inplace=True)

In [55]:
all_channels.head(2)

,destination,source,path_id,capacity,node1_fee_rate,node2_fee_rate,s_node,d_node
0,0298f6074a454a1f5345cb2a7c6f9fce206cd0bf675d17...,0305f5f4013f6c6eeb097bd8607204ec1f31577a05fae3...,0,5000000,249,100,LN.capital,BCash_Is_Trash
1,03434a39cd9a537c852fc8fb72454086d726f9111e9f73...,0298f6074a454a1f5345cb2a7c6f9fce206cd0bf675d17...,0,1000000,2,10,BCash_Is_Trash,***ROUTE 66***


In [56]:
df = all_channels.copy()

In [57]:
df[['s_node', 'd_node']] = df[['s_node', 'd_node']].fillna('NO_NODE_ALIAS')

In [58]:
df.fillna(0, inplace=True)

# DASH

In [59]:
from jupyter_dash import JupyterDash

In [60]:
from dash import Dash, dash_table, dcc, html

In [61]:
from dash.dependencies import Input, Output

In [62]:
app = JupyterDash(__name__)
server = app.server

app.layout = html.Div([
    dash_table.DataTable(
        id='potential-paths-table',
        columns=[
            {'name': i, 'id': i, 'deletable': True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False, # cells are not editable 
        filter_action='native', # filter by entering text in the input box
        sort_action='native', # sort by click
        sort_mode='multi', # multi-column sorting
        row_selectable='multi', # multiple rows can be selected
        row_deletable=False,
        page_action='native',
        page_current=0,
        page_size=15,
    ),
    html.Div(id='datatable-container'),
])

In [63]:
app.run_server(mode="inline")

## Idea: table with paths packed in lists.

Make a dataframe where each row is a path contained in a list. 

The fees, and channel capacities are the aggregate of the fees and capacities of the channels in the path.

This way, we can easily compare the fees and capacities of different paths. We can make an interactive table to filter, or sort the paths. Then, when selecting one path, another table will show the different channels in the path with their fees and capacities.

In [64]:
df.head(2)

,destination,source,path_id,capacity,node1_fee_rate,node2_fee_rate,s_node,d_node
0,0298f6074a454a1f5345cb2a7c6f9fce206cd0bf675d17...,0305f5f4013f6c6eeb097bd8607204ec1f31577a05fae3...,0,5000000,249,100,LN.capital,BCash_Is_Trash
1,03434a39cd9a537c852fc8fb72454086d726f9111e9f73...,0298f6074a454a1f5345cb2a7c6f9fce206cd0bf675d17...,0,1000000,2,10,BCash_Is_Trash,***ROUTE 66***


In [65]:
dff = df.copy()

In [66]:
# fee rates should be int
dff[['node1_fee_rate', 'node2_fee_rate']] = dff[['node1_fee_rate', 'node2_fee_rate']].astype(np.int64)

In [67]:
# make a dataframe where each row is a path contained in a list

grouped_routes = dff.groupby('path_id').apply(lambda x: x.s_node.values.tolist() + [x.d_node.values.tolist()[-1]])

In [68]:
grouped_routes.tail(2)

path_id
135319    [LN.capital, ln2me.com / LightningTo.Me, Korti...
135320    [LN.capital, ln2me.com / LightningTo.Me, Korti...
dtype: object

In [69]:
grouped_routes = grouped_routes.to_frame().rename(columns={0: 'route'})

In [70]:
# join aggregated capacity and fee rates to each path
grouped_routes = grouped_routes.join(dff.groupby('path_id').aggregate(sum))

In [71]:
# add peers column ('first_node - last_node')
grouped_routes['peers'] = grouped_routes.route.apply(lambda x: f'{x[0]} - {x[-1]}')

In [72]:
grouped_routes.head(2)

,route,capacity,node1_fee_rate,node2_fee_rate,peers
path_id,,,,,
0,"[LN.capital, BCash_Is_Trash, ***ROUTE 66***]",6000000,251,110,LN.capital - ***ROUTE 66***
1,"[LN.capital, southxchange.com, ***ROUTE 66***]",10550000,200,110,LN.capital - ***ROUTE 66***


In [73]:
# add a 'hops' column that represents the number of hops in each path
grouped_routes['hops'] = grouped_routes['route'].apply(lambda x: len(x) - 1)

In [74]:
grouped_routes.route

path_id
0              [LN.capital, BCash_Is_Trash, ***ROUTE 66***]
1            [LN.capital, southxchange.com, ***ROUTE 66***]
2                   [LN.capital, Milky Way, ***ROUTE 66***]
3         [LN.capital, ln2me.com / LightningTo.Me, south...
4         [LN.capital, ln2me.com / LightningTo.Me, ACINQ...
                                ...                        
135316    [LN.capital, ln2me.com / LightningTo.Me, Korti...
135317    [LN.capital, ln2me.com / LightningTo.Me, Korti...
135318    [LN.capital, ln2me.com / LightningTo.Me, Korti...
135319    [LN.capital, ln2me.com / LightningTo.Me, Korti...
135320    [LN.capital, ln2me.com / LightningTo.Me, Korti...
Name: route, Length: 135321, dtype: object

In [75]:
# change route column to be str instead of list
grouped_routes['route'] = grouped_routes['route'].apply(lambda x: ' -> '.join(x))

In [76]:
grouped_routes.head(2)

,route,capacity,node1_fee_rate,node2_fee_rate,peers,hops
path_id,,,,,,
0,LN.capital -> BCash_Is_Trash -> ***ROUTE 66***,6000000,251,110,LN.capital - ***ROUTE 66***,2
1,LN.capital -> southxchange.com -> ***ROUTE 66***,10550000,200,110,LN.capital - ***ROUTE 66***,2


In [77]:
grouped_routes.insert(loc=0, column="id", value=grouped_routes.index)

In [78]:
grouped_routes.head(2)

,id,route,capacity,node1_fee_rate,node2_fee_rate,peers,hops
path_id,,,,,,,
0,0,LN.capital -> BCash_Is_Trash -> ***ROUTE 66***,6000000,251,110,LN.capital - ***ROUTE 66***,2
1,1,LN.capital -> southxchange.com -> ***ROUTE 66***,10550000,200,110,LN.capital - ***ROUTE 66***,2


In [79]:
### DATASET FOR THE FIRST TABLE
# Should contain the following columns:
# source_node, destination_node, peers
# the index should be same as peers
# unique_sources, unique_destinations = [], []

# unique sources
unique_sources = [x.split(' - ')[0] for x in grouped_routes.peers.unique().tolist()]


# unique destinations
unique_destinations = [x.split(' - ')[-1] for x in grouped_routes.peers.unique().tolist() ]

In [80]:
options_df = pd.DataFrame(
    data={'source':unique_sources, 'destination':unique_destinations}, 
    index=grouped_routes.peers.unique()
)

In [81]:
options_df.insert(loc=0, column="id", value=options_df.index)
options_df

,id,source,destination
LN.capital - ***ROUTE 66***,LN.capital - ***ROUTE 66***,LN.capital,***ROUTE 66***
LN.capital - cybertide.com,LN.capital - cybertide.com,LN.capital,cybertide.com
LN.capital - hmmmstrange,LN.capital - hmmmstrange,LN.capital,hmmmstrange
LN.capital - ln.satjar.com,LN.capital - ln.satjar.com,LN.capital,ln.satjar.com
LN.capital - AQUARIOUS,LN.capital - AQUARIOUS,LN.capital,AQUARIOUS
...,...,...,...
LN.capital - Holtzman,LN.capital - Holtzman,LN.capital,Holtzman
LN.capital - Libertas,LN.capital - Libertas,LN.capital,Libertas
LN.capital - C-Nugget,LN.capital - C-Nugget,LN.capital,C-Nugget
LN.capital - Greasedlightning,LN.capital - Greasedlightning,LN.capital,Greasedlightning


In [82]:
app = JupyterDash(__name__)
server = app.server

###############################################
#               FIRST TABLE                   #
###############################################

app.layout = html.Div([
    dash_table.DataTable(
        id='sourceDest-table',
        columns=[
            {'name': i, 'id': i, 'deletable': True} for i in options_df.columns
            if i != 'id'
        ],
        data=options_df.to_dict('records'),
        editable=False, # cells are not editable
        filter_action='native', # filter by entering text in the input box
        row_selectable='single',  
        page_action='native',
        page_current=0,
        page_size=6,
    ),
    html.Div(id='secondtable-container'),  
    html.Div(id="output") 
])


###############################################
#               Second table                  #
###############################################

@app.callback(
    Output('secondtable-container', 'children'),
    Output('output', 'children'),
    Input('sourceDest-table', 'selected_row_ids')
)
def update_secondtable(selected_rows):
    # app.layout = html.Div([
    if selected_rows is None:
        rows = []
    
    second_dff = grouped_routes[grouped_routes.peers.isin(selected_rows)]
    return html.Div([
        dash_table.DataTable(
            id='potential-paths-table',
            columns=[
                {'name': i, 'id': i, 'deletable': True} for i in second_dff.columns
                if i not in ['id', 's_node', 'd_node']
            ],
            style_cell={
                'overflow': 'hidden',
                'textOverflow': 'ellipsis',
                'maxWidth': 0,
            },

            tooltip_data=[
            {
                column: {'value': str(value), 'type': 'markdown'}
                for column, value in row.items()
            } for row in second_dff.to_dict('records')
            ],
            tooltip_duration=None,

            data=second_dff.to_dict('records'),
            editable=False, # cells are not editable 
            filter_action='native', # filter by entering text in the input box
            sort_action='native', # sort by click
            sort_mode='multi', # multi-column sorting
            row_selectable='single', # multiple rows can be selected
            row_deletable=False,
            page_action='native',
            page_current=0,
            page_size=6,
        ),
        html.Div(id='datatable-container'),
        html.Div(id='output2')
    ]), f'Row ids: {selected_rows}'
    

###############################################
#               Third table                   #
###############################################

@app.callback(
    Output('datatable-container', 'children'),
    Output('output2', 'children'),
    Input('potential-paths-table', 'selected_row_ids'),
)
def make_specific_path_table(row_ids):
    if row_ids is None:
        row_ids = []
    
    dff = df[df.path_id.isin(row_ids)]

    return dash_table.DataTable(
        id='specific-paths-table',
        columns=[
            {'name': i, 'id': i, 'deletable': True} for i in dff.columns
            # Omit path_id column
            if i != 'path_id'
        ],

        style_table={'overflowX': 'auto'},
        style_cell={
            # all three widths are needed
            'minWidth': '180px', 'width': '180px', 'maxWidth': '180px',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
        },

        tooltip_data=[
        {
            column: {'value': str(value), 'type': 'markdown'}
            for column, value in row.items()
        } for row in df.to_dict('records')
        ],
        tooltip_duration=None,

        data=dff.to_dict('records'),
        editable=False, # cells are not editable
        filter_action='native', # filter by entering text in the input box
        sort_action='native', # sort by click
        sort_mode='multi', # multi-column sorting
        row_deletable=False,

    ), f'Row ids: {row_ids}'
    

In [83]:
app.run_server(mode="inline")

# Refactor the above code to adapt it for rebalancing purposes.

**Idea**

4 tables:

1. Table to select source node (our node), central node, and the channel (channel out) that helps rebalance another channel (channel in).
2. Table to select a path from the possible paths to get to the central node.
3. Table to select channel in.
4. Table to select a path from the possible paths to complete the rebalance (circular path).

NOTE: At the end, another table could show the entire ciruclar path, with all the channels information.

In [84]:
from jupyter_dash import JupyterDash
from dash import Dash, dash_table, dcc, html
from dash.dependencies import Input, Output

## 1st table (source node, central node, channel out)

In [85]:
# OUR NODE | CENTRAL NODE | CHANNEL OUT

In [86]:
grouped_routes.head(3)

,id,route,capacity,node1_fee_rate,node2_fee_rate,peers,hops
path_id,,,,,,,
0,0,LN.capital -> BCash_Is_Trash -> ***ROUTE 66***,6000000,251,110,LN.capital - ***ROUTE 66***,2
1,1,LN.capital -> southxchange.com -> ***ROUTE 66***,10550000,200,110,LN.capital - ***ROUTE 66***,2
2,2,LN.capital -> Milky Way -> ***ROUTE 66***,11000000,110,10050,LN.capital - ***ROUTE 66***,2


In [87]:
grouped_routes.shape

(135321, 7)

In [88]:
# Create a new dataframe where for each route retrieve the source and destination nodes, and the channel out (the second node)
select_central_node_df = pd.DataFrame(columns=['source', 'central_node', 'channel_out'], index=grouped_routes.index)

select_central_node_df['source'] = grouped_routes.peers.apply(lambda x: x.split(' - ')[0])
select_central_node_df['central_node'] = grouped_routes.peers.apply(lambda x: x.split(' - ')[-1])
select_central_node_df['channel_out'] = grouped_routes.route.apply(lambda x: x.split(' -> ')[1])

In [89]:
# thirds column
select_central_node_df['important_nodes'] = select_central_node_df.apply(lambda x: x.source + ' - ' + x.central_node + ' -> ' + x.channel_out, axis=1)

In [90]:
select_central_node_df['channel_out'] = select_central_node_df.source + ' - ' + select_central_node_df.channel_out

In [91]:
# drop duplicates from the three columns
select_central_node_df.drop_duplicates(subset='important_nodes', keep='first', inplace=True)

In [92]:
select_central_node_df.head(2)

,source,central_node,channel_out,important_nodes
path_id,,,,
0,LN.capital,***ROUTE 66***,LN.capital - BCash_Is_Trash,LN.capital - ***ROUTE 66*** -> BCash_Is_Trash
1,LN.capital,***ROUTE 66***,LN.capital - southxchange.com,LN.capital - ***ROUTE 66*** -> southxchange.com


In [93]:
select_central_node_df['id'] = select_central_node_df.important_nodes.values

In [94]:
app = JupyterDash(__name__)
server = app.server

app.layout = html.Div([
    dash_table.DataTable(
        id='first-table',
        columns=[
            {'name': i, 'id': i, 'deletable': True} for i in select_central_node_df.columns
            if i not in ['id', 'important_nodes']
        ],
        data=select_central_node_df.to_dict('records'),
        editable=False, # cells are not editable
        filter_action='native', # filter by entering text in the input box
        row_selectable='single',  
        page_action='native',
        page_current=0,
        page_size=6,
    ),
    html.Div(id='secondtable-container'),  
    html.Div(id="output") 
])

In [95]:
app.run_server()

Dash app running on http://127.0.0.1:8050/


## 2nd table (paths to central node)

In [96]:
grouped_routes['channel_out'] = grouped_routes.route.apply(lambda x: x.split(' -> ')[1])

In [97]:
app = JupyterDash(__name__)
server = app.server

app.layout = html.Div([
    dash_table.DataTable(
        id='first-table',
        columns=[
            {'name': i, 'id': i, 'deletable': True} for i in select_central_node_df.columns
            if i != 'id'
        ],
        data=select_central_node_df.to_dict('records'),
        editable=False, # cells are not editable
        filter_action='native', # filter by entering text in the input box
        row_selectable='single',  
        page_action='native',
        page_current=0,
        page_size=6,
    ),
    html.Div(id='secondtable-container'),  
    html.Div(id="output") 
])

@app.callback(
    Output('secondtable-container', 'children'),
    Input('first-table', 'selected_row_ids'),
)
def make_second_table(selected_rows):
    if selected_rows is None:
        rows = []
    else:
        rows = [row.split(' -> ') for row in selected_rows]
        source_central = [row[0] for row in rows]
        channel_out = [row[1] for row in rows]

    masks = (grouped_routes.peers.isin(source_central) & grouped_routes.channel_out.isin(channel_out))
    second_dff = grouped_routes[masks]

    return html.Div([
        dash_table.DataTable(
            id='second-table',
            columns=[
                {'name': i, 'id': i, 'deletable': True} for i in second_dff.columns
                if i != 'id'
            ],
            data=second_dff.to_dict('records'),
            editable=False, # cells are not editable
            filter_action='native', # filter by entering text in the input box
            row_selectable='single',  
            page_action='native',
            page_current=0,
            page_size=10,
        ),
        html.Div(id='thirdtable-container'),  
        html.Div(id="output2") 
    ])

In [98]:
app.run_server()

Dash app running on http://127.0.0.1:8050/


## Third table (to select channel in)

In [99]:
# paths from okcoin to ln_capital
paths_okcoin_ln_capital = []
for i in range(9):
    stop = 0
    paths = nx.all_simple_paths(G, source=okcoin, target=ln_capital, cutoff=i+1)
    while stop < 100:
        try:
            paths_okcoin_ln_capital.append(next(paths))
            stop += 1
        except StopIteration:
            break

In [100]:
test2 = pd.DataFrame(data={'testing':paths_okcoin_ln_capital})
testing = test2.explode('testing')
testing.reset_index(inplace=True)

testing = testing.rename(columns={'testing':'source'})
testing['destination'] = testing.source.shift(-1)

grouped2 = testing.groupby('index')
all_channels2 = testing[grouped2.cumcount(ascending=False) > 0]

# Need to add capacity and node fee rates
# make capacity and node fee rates columns
all_channels2[['capacity', 'node1_fee_rate', 'node2_fee_rate']] = None 
# iterate over all_channels2 rows and add capacity and node fee rates from G edges
for index, row in all_channels2.iterrows():
    all_channels2.loc[index, 'capacity'] = G.get_edge_data(row['source'], row['destination'])['capacity']
    all_channels2.loc[index, 'node1_fee_rate'] = G.get_edge_data(row['source'], row['destination'])['node1_fee_rate']
    all_channels2.loc[index, 'node2_fee_rate'] = G.get_edge_data(row['source'], row['destination'])['node2_fee_rate']
all_channels2 = all_channels2.rename(columns={'index':'path_id'})
aliases = nodes.set_index('pub_key')['alias']
all_channels2.set_index('source', inplace=True)
all_channels2['s_node'] = aliases
all_channels2.reset_index(inplace=True)
all_channels2.set_index('destination',inplace=True)
all_channels2['d_node'] = aliases
all_channels2.reset_index(inplace=True)
central_to_source_df = all_channels2.copy()
central_to_source_df[['s_node', 'd_node']] = central_to_source_df[['s_node', 'd_node']].fillna('NO_NODE_ALIAS')
central_to_source_df.fillna(0, inplace=True)

In [101]:
central_to_source_df.head(2)

,destination,source,path_id,capacity,node1_fee_rate,node2_fee_rate,s_node,d_node
0,0305f5f4013f6c6eeb097bd8607204ec1f31577a05fae3...,036b53093df5a932deac828cca6d663472dbc88322b05e...,0,10000000,790,500,okcoin,LN.capital
1,03cde60a6323f7122d5178255766e38114b4722ede08f7...,036b53093df5a932deac828cca6d663472dbc88322b05e...,1,500000000,500,1000,okcoin,bfx-lnd1


In [102]:
# fee rates to int
central_to_source_df['node1_fee_rate'] = central_to_source_df['node1_fee_rate'].astype(int)
central_to_source_df['node2_fee_rate'] = central_to_source_df['node2_fee_rate'].astype(int)

In [103]:
grouped_routes2 = central_to_source_df.groupby('path_id').apply(lambda x: x.s_node.values.tolist() + [x.d_node.values.tolist()[-1]])
grouped_routes2 = grouped_routes2.to_frame().rename(columns={0: 'route'})
# join aggregated capacity and fee rates to each path
grouped_routes2 = grouped_routes2.join(central_to_source_df.groupby('path_id').aggregate(sum))
# add peers column ('first_node - last_node')
grouped_routes2['peers'] = grouped_routes2.route.apply(lambda x: f'{x[0]} - {x[-1]}')
# add a 'hops' column that represents the number of hops in each path
grouped_routes2['hops'] = grouped_routes2['route'].apply(lambda x: len(x) - 1)
# change route column to be str instead of list
grouped_routes2['route'] = grouped_routes2['route'].apply(lambda x: ' -> '.join(x))
grouped_routes2.insert(loc=0, column="id", value=grouped_routes2.index)

In [104]:
grouped_routes2.head(2)

,id,route,capacity,node1_fee_rate,node2_fee_rate,peers,hops
path_id,,,,,,,
0,0,okcoin -> LN.capital,10000000,790,500,okcoin - LN.capital,1
1,1,okcoin -> bfx-lnd1 -> LN.capital,507000000,1290,2000,okcoin - LN.capital,2


In [105]:

# Create a new dataframe where for each route retrieve the source and destination nodes, and the channel out (the second node)
select_channel_in_df = pd.DataFrame(columns=['central_node', 'source', 'channel_in'], index=grouped_routes2.index)

select_channel_in_df['central_node'] = grouped_routes2.peers.apply(lambda x: x.split(' - ')[0])
select_channel_in_df['source'] = grouped_routes2.peers.apply(lambda x: x.split(' - ')[-1])
select_channel_in_df['channel_in'] = grouped_routes2.route.apply(lambda x: x.split(' -> ')[-2])
# thirds column
select_channel_in_df['important_nodes'] = select_channel_in_df.apply(lambda x: x.central_node + ' - ' +  x.channel_in + ' - ' + x.source, axis=1)
select_channel_in_df['channel_in'] = select_channel_in_df.channel_in + ' - ' + select_channel_in_df.source
# drop duplicates from the three columns
select_channel_in_df.drop_duplicates(subset='important_nodes', keep='first', inplace=True)
select_channel_in_df['id'] = select_channel_in_df.important_nodes.values

In [106]:
select_channel_in_df

,central_node,source,channel_in,important_nodes,id
path_id,,,,,
0,okcoin,LN.capital,okcoin - LN.capital,okcoin - okcoin - LN.capital,okcoin - okcoin - LN.capital
1,okcoin,LN.capital,bfx-lnd1 - LN.capital,okcoin - bfx-lnd1 - LN.capital,okcoin - bfx-lnd1 - LN.capital
2,okcoin,LN.capital,bfx-lnd0 - LN.capital,okcoin - bfx-lnd0 - LN.capital,okcoin - bfx-lnd0 - LN.capital
3,okcoin,LN.capital,okex - LN.capital,okcoin - okex - LN.capital,okcoin - okex - LN.capital
4,okcoin,LN.capital,BCash_Is_Trash - LN.capital,okcoin - BCash_Is_Trash - LN.capital,okcoin - BCash_Is_Trash - LN.capital
...,...,...,...,...,...
261,okcoin,LN.capital,MJKC31 - LN.capital,okcoin - MJKC31 - LN.capital,okcoin - MJKC31 - LN.capital
331,okcoin,LN.capital,02415d7ec37a7bb22b27 - LN.capital,okcoin - 02415d7ec37a7bb22b27 - LN.capital,okcoin - 02415d7ec37a7bb22b27 - LN.capital
546,okcoin,LN.capital,AVATAR - LN.capital,okcoin - AVATAR - LN.capital,okcoin - AVATAR - LN.capital


In [107]:
app.layout = html.Div([
    dash_table.DataTable(
        id='third-table',
        columns=[
            {'name': i, 'id': i, 'deletable': True} for i in select_channel_in_df.columns
            if i != 'id'
        ],
        data=select_channel_in_df.to_dict('records'),
        editable=False, # cells are not editable
        filter_action='native', # filter by entering text in the input box
        row_selectable='single',  
        page_action='native',
        page_current=0,
        page_size=6,
    ),
    html.Div(id='fourthtable-container'),  
    html.Div(id="output3") 
])

In [108]:
app.run_server()

Dash app running on http://127.0.0.1:8050/


## Fourth table (path to complete the rebalance)

In [109]:
grouped_routes2['channel_in'] = grouped_routes2.route.apply(lambda x: x.split(' -> ')[-2])

In [110]:
grouped_routes2.head(2)

,id,route,capacity,node1_fee_rate,node2_fee_rate,peers,hops,channel_in
path_id,,,,,,,,
0,0,okcoin -> LN.capital,10000000,790,500,okcoin - LN.capital,1,okcoin
1,1,okcoin -> bfx-lnd1 -> LN.capital,507000000,1290,2000,okcoin - LN.capital,2,bfx-lnd1


In [111]:
app = JupyterDash(__name__)
server = app.server

app.layout = html.Div([
    dash_table.DataTable(
        id='third-table',
        columns=[
            {'name': i, 'id': i, 'deletable': True} for i in select_channel_in_df.columns
            if i != 'id'
        ],
        data=select_channel_in_df.to_dict('records'),
        editable=False, # cells are not editable
        filter_action='native', # filter by entering text in the input box
        row_selectable='single',  
        page_action='native',
        page_current=0,
        page_size=6,
    ),
    html.Div(id='fourthtable-container'),  
    html.Div(id="output3") 
])

@app.callback(
    Output('fourthtable-container', 'children'),
    Input('third-table', 'selected_row_ids')
)
def make_fourth_table(selected_rows):
    if selected_rows is None:
        central_source, chan_in = [], []
    else:
        rows = [row.split(' - ') for row in selected_rows]
        central = [row[0] for row in rows]
        chan_in = [row[1] for row in rows]
        source = [row[-1] for row in rows]
        central_source = [central[i] + ' - ' + source[i] for i in range(len(central))]

    masks = (grouped_routes2.peers.isin(central_source) & grouped_routes2.channel_in.isin(chan_in))
    second_dff = grouped_routes2[masks]

    return html.Div([
        dash_table.DataTable(
            id='second-table',
            columns=[
                {'name': i, 'id': i, 'deletable': True} for i in second_dff.columns
                if i != 'id'
            ],
            data=second_dff.to_dict('records'),
            editable=False, # cells are not editable
            filter_action='native', # filter by entering text in the input box
            row_selectable='single',  
            page_action='native',
            page_current=0,
            page_size=10,
        ),
    ])


In [112]:
app.run_server()

Dash app running on http://127.0.0.1:8050/


## Join all tables together

In [113]:
app = JupyterDash(__name__)
server = app.server

### FIRST TABLE ###

app.layout = html.Div([
    dash_table.DataTable(
        id='first-table',
        columns=[
            {'name': i, 'id': i, 'deletable': True} for i in select_central_node_df.columns
            if i != 'id' and i != 'important_nodes'
        ],
        data=select_central_node_df.to_dict('records'),
        editable=False, # cells are not editable
        filter_action='native', # filter by entering text in the input box
        row_selectable='single',  
        page_action='native',
        page_current=0,
        page_size=6,
    ),
    html.Div(id='secondtable-container'),  
])

### SECOND TABLE ###

@app.callback(
    Output('secondtable-container', 'children'),
    Input('first-table', 'selected_row_ids'),
)
def make_second_table(selected_rows):
    if selected_rows is None:
        source_central, channel_out = [], []
    else:
        rows = [row.split(' -> ') for row in selected_rows]
        source_central = [row[0] for row in rows]
        channel_out = [row[1] for row in rows]

    masks = (grouped_routes.peers.isin(source_central) & grouped_routes.channel_out.isin(channel_out))
    second_dff = grouped_routes[masks]

    return html.Div([
        dash_table.DataTable(
            id='second-table',
            columns=[
                {'name': i, 'id': i, 'deletable': True} for i in second_dff.columns
                if i != 'id' and i != 'channel_out'
            ],
            data=second_dff.to_dict('records'),
            editable=False, # cells are not editable
            filter_action='native', # filter by entering text in the input box
            sort_action='native',
            sort_mode='multi',
            row_selectable='single',  
            page_action='native',
            page_current=0,
            page_size=10,
        ),
        html.Div(id='thirdtable-container'),  
    ])

### THIRD TABLE ###

@app.callback(
    Output('thirdtable-container', 'children'),
    Input('second-table', 'selected_row_ids'),
)
def make_third_table(selected_rows):
    if selected_rows is None:
        rows = []
    else:
        rows = select_channel_in_df
    return html.Div([
    dash_table.DataTable(
        id='third-table',
        columns=[
            {'name': i, 'id': i, 'deletable': True} for i in rows.columns
            if i != 'id' and i != 'important_nodes'
        ],
        data=rows.to_dict('records'),
        editable=False, # cells are not editable
        filter_action='native', # filter by entering text in the input box
        row_selectable='single',  
        page_action='native',
        page_current=0,
        page_size=6,
    ),
    html.Div(id='fourthtable-container'),  
])

### FOURTH TABLE ###

@app.callback(
    Output('fourthtable-container', 'children'),
    Input('third-table', 'selected_row_ids')
)
def make_fourth_table(selected_rows):
    if selected_rows is None:
        rows = []
    else:
        rows = [row.split(' - ') for row in selected_rows]
        central = [row[0] for row in rows]
        chan_in = [row[1] for row in rows]
        source = [row[-1] for row in rows]
        central_source = [central[i] + ' - ' + source[i] for i in range(len(central))]

    masks = (grouped_routes2.peers.isin(central_source) & grouped_routes2.channel_in.isin(chan_in))
    second_dff = grouped_routes2[masks]

    return html.Div([
    dash_table.DataTable(
            id='fourth-table',
            columns=[
                {'name': i, 'id': i, 'deletable': True} for i in second_dff.columns
                if i != 'id' and i != 'channel_in'
            ],
            data=second_dff.to_dict('records'),
            editable=False, # cells are not editable
            filter_action='native', # filter by entering text in the input box
            sort_action='native',
            sort_mode='multi',
            row_selectable='single',  
            page_action='native',
            page_current=0,
            page_size=10,
        ),
        html.Div(id='final-container'),
    ])

### FINAL CONTAINER ###
# this is for displaying all the circular path of the rebalancing, with all its data #

@app.callback(
    Output('final-container', 'children'),
   
    Input('second-table', 'selected_row_ids'),
    Input('fourth-table', 'selected_row_ids'),
)
def fill_final_container(second_rows, fourth_rows):
    if second_rows is None:
        rows = []
    else:
        rows = second_rows

    dff = grouped_routes.loc[rows]

    if fourth_rows is None:
        f_rows = []
    else:
        f_rows = fourth_rows
    
    f_dff = grouped_routes2.loc[f_rows]

    circular_path = dff.route.values[0]

    chan_in_path = f_dff.route.values[0]
    
    circular_path = circular_path + chan_in_path[chan_in_path.find(' -> '):]

    aggregated_capacities = dff.capacity.values[0] + f_dff.capacity.values[0]
    aggregated_fees = dff.node2_fee_rate.values[0] + f_dff.node2_fee_rate.values[0]
    total_hops = dff.hops.values[0] + f_dff.hops.values[0]


    return html.Div([
        html.B(circular_path),
        html.Br(),
        html.B(f'Channel agg capacities: {aggregated_capacities}'),
        html.Br(),
        html.B(f'Channel agg fees: {aggregated_fees}'),
        html.Br(),
        html.B(f'Total hops in path: {total_hops}'),
    ])
    

In [114]:
app.run_server()

Dash app running on http://127.0.0.1:8050/


In [115]:
grouped_routes2

,id,route,capacity,node1_fee_rate,node2_fee_rate,peers,hops,channel_in
path_id,,,,,,,,
0,0,okcoin -> LN.capital,10000000,790,500,okcoin - LN.capital,1,okcoin
1,1,okcoin -> bfx-lnd1 -> LN.capital,507000000,1290,2000,okcoin - LN.capital,2,bfx-lnd1
2,2,okcoin -> bfx-lnd0 -> LN.capital,204000000,1100,1500,okcoin - LN.capital,2,bfx-lnd0
3,3,okcoin -> okex -> LN.capital,107000000,10000,100,okcoin - LN.capital,2,okex
4,4,okcoin -> BCash_Is_Trash -> LN.capital,21777215,251,600,okcoin - LN.capital,2,BCash_Is_Trash
...,...,...,...,...,...,...,...,...
711,711,okcoin -> bfx-lnd1 -> BCash_Is_Trash -> tippin...,649302845,38625,4521,okcoin - LN.capital,9,Kraken 🐙⚡
712,712,okcoin -> bfx-lnd1 -> BCash_Is_Trash -> tippin...,546302845,72885,2176,okcoin - LN.capital,9,Mintter
713,713,okcoin -> bfx-lnd1 -> BCash_Is_Trash -> tippin...,557080060,17541,2186,okcoin - LN.capital,9,ln.bitstamp.net [Bitstamp]
